In [12]:
import pandas as pd
import random
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Constants
NUM_TASKS = 1000
NUM_SERVERS = 3
NUM_DEVICES = 10  # Multiple devices

In [13]:
# Generate synthetic dataset for tasks
def generate_tasks(num_tasks):
    data = []
    for task_id in range(num_tasks):
        complexity = random.randint(1, 10)
        resources_needed = random.randint(1, 3)
        priority = random.randint(1, 10)
        deadline = random.randint(5, 30)  # Deadline in seconds
        data_size = random.randint(1, 100)  # Data size in MB
        data.append([task_id, complexity, resources_needed, priority, deadline, data_size])
    return pd.DataFrame(data, columns=['TaskID', 'Complexity', 'ResourcesNeeded', 'Priority', 'Deadline', 'DataSize'])


In [14]:
# Generate synthetic dataset for servers
def generate_servers(num_servers):
    data = []
    for server_id in range(num_servers):
        capacity = random.randint(5, 15)
        load = random.uniform(0, 1)  # Load on server (0 to 1)
        power_consumption = random.randint(100, 200)  # Power consumption in watts
        network_latency = random.uniform(0.1, 2.0)  # Network latency in seconds
        cpu = random.uniform(2.0, 4.0)  # CPU in GHz
        memory = random.randint(8, 32)  # Memory in GB
        storage = random.randint(100, 1000)  # Storage in GB
        bandwidth = random.uniform(10.0, 100.0)  # Bandwidth in Mbps
        data.append([server_id, capacity, load, power_consumption, network_latency, cpu, memory, storage, bandwidth])
    return pd.DataFrame(data, columns=['ServerID', 'Capacity', 'Load', 'PowerConsumption', 'NetworkLatency',
                                       'CPU', 'Memory', 'Storage', 'Bandwidth'])


In [15]:
# Generate synthetic dataset for devices
def generate_devices(num_devices):
    data = []
    for device_id in range(num_devices):
        cpu = random.uniform(1.0, 3.0)  # CPU in GHz
        memory = random.randint(4, 16)  # Memory in GB
        storage = random.randint(50, 500)  # Storage in GB
        battery = random.randint(20, 100)  # Battery percentage
        power_consumption = random.randint(5, 15)  # Power consumption in watts
        data.append([device_id, cpu, memory, storage, battery, power_consumption])
    return pd.DataFrame(data, columns=['DeviceID', 'CPU', 'Memory', 'Storage', 'Battery', 'PowerConsumption'])


In [16]:
# Combine datasets into a single dataset for training
def combine_datasets(tasks, servers, devices):
    combined_data = []
    for _, task in tasks.iterrows():
        for _, server in servers.iterrows():
            for _, device in devices.iterrows():
                task_id = task['TaskID']
                complexity = task['Complexity']
                resources_needed = task['ResourcesNeeded']
                priority = task['Priority']
                deadline = task['Deadline']
                data_size = task['DataSize']
                
                server_id = server['ServerID']
                server_capacity = server['Capacity']
                server_load = server['Load']
                server_power_consumption = server['PowerConsumption']
                server_latency = server['NetworkLatency']
                server_cpu = server['CPU']
                server_memory = server['Memory']
                server_storage = server['Storage']
                server_bandwidth = server['Bandwidth']
                
                device_cpu = device['CPU']
                device_memory = device['Memory']
                device_storage = device['Storage']
                device_battery = device['Battery']
                device_power_consumption = device['PowerConsumption']
                
                # Determine offloading decision
                if (complexity > 5 and  # Heuristic for complexity
                    resources_needed <= server_capacity and
                    server_load < 0.7 and
                    priority > 5 and
                    deadline > server_latency and
                    data_size <= server_storage and
                    device_cpu >= server_cpu and
                    device_memory >= server_memory):
                    offload = 1
                else:
                    offload = 0
                
                combined_data.append([task_id, complexity, resources_needed, priority, deadline, data_size,
                                      server_id, server_capacity, server_load, server_power_consumption, server_latency,
                                      server_cpu, server_memory, server_storage, server_bandwidth,
                                      device_cpu, device_memory, device_storage, device_battery, device_power_consumption, offload])
    
    return pd.DataFrame(combined_data, columns=['TaskID', 'Complexity', 'ResourcesNeeded', 'Priority', 'Deadline', 'DataSize',
                                                'ServerID', 'ServerCapacity', 'ServerLoad', 'ServerPowerConsumption', 'ServerLatency',
                                                'ServerCPU', 'ServerMemory', 'ServerStorage', 'ServerBandwidth',
                                                'DeviceCPU', 'DeviceMemory', 'DeviceStorage', 'DeviceBattery', 'DevicePowerConsumption', 'Offload'])


In [6]:
# Create datasets
tasks = generate_tasks(NUM_TASKS)
servers = generate_servers(NUM_SERVERS)
devices = generate_devices(NUM_DEVICES)

# Print datasets for verification
print("Tasks Dataset:")
print(tasks.head())
print("\nServers Dataset:")
print(servers.head())
print("\nDevices Dataset:")
print(devices.head())

Tasks Dataset:
   TaskID  Complexity  ResourcesNeeded  Priority  Deadline  DataSize
0       0           5                2         6        13        19
1       1           4                3         8        29        29
2       2           4                3         8        15         7
3       3          10                3        10        19         6
4       4           6                2        10        14         9

Servers Dataset:
   ServerID  Capacity      Load  PowerConsumption  NetworkLatency       CPU  \
0         0        12  0.146221               113        1.232060  2.316920   
1         1         7  0.671067               128        1.539642  2.637980   
2         2         5  0.270808               137        1.740329  2.139088   

   Memory  Storage  Bandwidth  
0      13      756  66.493270  
1       8      895  95.269601  
2      28      280  97.292675  

Devices Dataset:
   DeviceID       CPU  Memory  Storage  Battery  PowerConsumption
0         0  2.139379   

In [7]:
# Combine datasets into a single dataset for training
def combine_datasets(tasks, servers, devices):
    combined_data = []
    for _, task in tasks.iterrows():
        for _, server in servers.iterrows():
            for _, device in devices.iterrows():
                task_id = task['TaskID']
                complexity = task['Complexity']
                resources_needed = task['ResourcesNeeded']
                priority = task['Priority']
                deadline = task['Deadline']
                data_size = task['DataSize']

                server_id = server['ServerID']
                server_capacity = server['Capacity']
                server_load = server['Load']
                server_power_consumption = server['PowerConsumption']
                server_latency = server['NetworkLatency']
                server_cpu = server['CPU']
                server_memory = server['Memory']
                server_storage = server['Storage']
                server_bandwidth = server['Bandwidth']

                device_cpu = device['CPU']
                device_memory = device['Memory']
                device_storage = device['Storage']
                device_battery = device['Battery']
                device_power_consumption = device['PowerConsumption']

                # Determine offloading decision
                if (complexity > 5 and  # Heuristic for complexity
                    resources_needed <= server_capacity and
                    server_load < 0.7 and
                    priority > 5 and
                    deadline > server_latency and
                    data_size <= server_storage and
                    device_cpu >= server_cpu and
                    device_memory >= server_memory):
                    offload = 1
                else:
                    offload = 0

                combined_data.append([task_id, complexity, resources_needed, priority, deadline, data_size,
                                      server_id, server_capacity, server_load, server_power_consumption, server_latency,
                                      server_cpu, server_memory, server_storage, server_bandwidth,
                                      device_cpu, device_memory, device_storage, device_battery, device_power_consumption, offload])

    return pd.DataFrame(combined_data, columns=['TaskID', 'Complexity', 'ResourcesNeeded', 'Priority', 'Deadline', 'DataSize',
                                                'ServerID', 'ServerCapacity', 'ServerLoad', 'ServerPowerConsumption', 'ServerLatency',
                                                'ServerCPU', 'ServerMemory', 'ServerStorage', 'ServerBandwidth',
                                                'DeviceCPU', 'DeviceMemory', 'DeviceStorage', 'DeviceBattery', 'DevicePowerConsumption', 'Offload'])


In [17]:
# Create datasets
tasks = generate_tasks(NUM_TASKS)
servers = generate_servers(NUM_SERVERS)
devices = generate_devices(NUM_DEVICES)

# Create combined dataset
combined_data = combine_datasets(tasks, servers, devices)

# Split into features and labels
X = combined_data[['Complexity', 'ResourcesNeeded', 'Priority', 'Deadline', 'DataSize',
                   'ServerCapacity', 'ServerLoad', 'ServerPowerConsumption', 'ServerLatency',
                   'ServerCPU', 'ServerMemory', 'ServerStorage', 'ServerBandwidth',
                   'DeviceCPU', 'DeviceMemory', 'DeviceStorage', 'DeviceBattery', 'DevicePowerConsumption']]
y = combined_data['Offload']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
# Initialize and train the decision tree classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6000

    accuracy                           1.00      6000
   macro avg       1.00      1.00      1.00      6000
weighted avg       1.00      1.00      1.00      6000



In [30]:
# Example of predicting whether to offload a new task with updated parameters
new_task = [[7, 8, 8, 20, 50,  # Task parameters: Complexity, ResourcesNeeded, Priority, Deadline, DataSize
             12, 0.4, 150, 1.2,  # Server parameters: ServerCapacity, ServerLoad, ServerPowerConsumption, ServerLatency
             3.5, 16, 600, 50,  # Server parameters: ServerCPU, ServerMemory, ServerStorage, ServerBandwidth
             1, 12, 300, 85, 10]]  # Device parameters: DeviceCPU, DeviceMemory, DeviceStorage, DeviceBattery, DevicePowerConsumption

# Predict offloading decision
offload_decision = clf.predict(new_task)

print("Offload Decision:", "Yes" if offload_decision[0] == 1 else "No")


Offload Decision: No


c:\Users\Aneesh Raskar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [21]:
# Testing function to generate data at random intervals
def generate_random_task():
    """Generate random task data."""
    complexity = random.randint(1, 10)
    resources_needed = random.randint(1, 3)
    priority = random.randint(1, 10)
    deadline = random.randint(5, 30)  # Deadline in seconds
    data_size = random.randint(1, 100)  # Data size in MB
    return [complexity, resources_needed, priority, deadline, data_size]

In [22]:
def generate_random_server():
    """Generate random server data."""
    server_capacity = random.randint(5, 15)
    server_load = random.uniform(0, 1)  # Load on server (0 to 1)
    server_power_consumption = random.randint(100, 200)  # Power consumption in watts
    server_latency = random.uniform(0.1, 2.0)  # Network latency in seconds
    server_cpu = random.uniform(2.0, 4.0)  # CPU in GHz
    server_memory = random.randint(8, 32)  # Memory in GB
    server_storage = random.randint(100, 1000)  # Storage in GB
    server_bandwidth = random.uniform(10.0, 100.0)  # Bandwidth in Mbps
    return [server_capacity, server_load, server_power_consumption, server_latency,
            server_cpu, server_memory, server_storage, server_bandwidth]

In [23]:
def generate_random_device():
    """Generate random device data."""
    device_cpu = random.uniform(1.0, 3.0)  # CPU in GHz
    device_memory = random.randint(4, 16)  # Memory in GB
    device_storage = random.randint(50, 500)  # Storage in GB
    device_battery = random.randint(20, 100)  # Battery percentage
    device_power_consumption = random.randint(5, 15)  # Power consumption in watts
    return [device_cpu, device_memory, device_storage, device_battery, device_power_consumption]

In [25]:
def generate_and_predict():
    """Generate random data and use the model to predict."""
    # Generate random data
    task_params = generate_random_task()
    server_params = generate_random_server()
    device_params = generate_random_device()
    
    # Combine data into the format required by the model
    new_task = [task_params[0], task_params[1], task_params[2], task_params[3], task_params[4],  # Task parameters
                server_params[0], server_params[1], server_params[2], server_params[3],  # Server parameters
                server_params[4], server_params[5], server_params[6], server_params[7],  # Server parameters
                device_params[0], device_params[1], device_params[2], device_params[3], device_params[4]]  # Device parameters
    
    # Make a prediction
    offload_decision = clf.predict([new_task])
    
    # Print results
    print(f"Task Parameters: {task_params}")
    print(f"Server Parameters: {server_params}")
    print(f"Device Parameters: {device_params}")
    print("Offload Decision:", "Yes" if offload_decision[0] == 1 else "No")

In [26]:
def test_model(interval_range=(1, 5), duration=30):
    """Generate data at random intervals for a specified duration."""
    start_time = time.time()
    while time.time() - start_time < duration:
        generate_and_predict()
        # Sleep for a random interval between interval_range[0] and interval_range[1] seconds
        time.sleep(random.uniform(interval_range[0], interval_range[1]))

In [27]:
# Run the testing function
test_model()

c:\Users\Aneesh Raskar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Task Parameters: [1, 2, 6, 7, 81]
Server Parameters: [12, 0.03111619387058373, 148, 1.4907608245374648, 3.0596077242325364, 13, 358, 58.22766296163326]
Device Parameters: [1.3971798561624151, 15, 271, 91, 11]
Offload Decision: No


c:\Users\Aneesh Raskar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Task Parameters: [10, 2, 2, 10, 6]
Server Parameters: [11, 0.25802437368112274, 159, 1.8959608969463304, 2.1898709284001825, 28, 165, 86.16715460590743]
Device Parameters: [1.16391276470558, 6, 415, 60, 7]
Offload Decision: No


c:\Users\Aneesh Raskar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


...

c:\Users\Aneesh Raskar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Task Parameters: [6, 3, 6, 29, 27]
Server Parameters: [7, 0.8067123252133386, 165, 0.527241723894413, 2.116072024012713, 27, 518, 50.708339970145325]
Device Parameters: [2.080138833796988, 8, 92, 47, 7]
Offload Decision: No


c:\Users\Aneesh Raskar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Task Parameters: [7, 1, 3, 22, 56]
Server Parameters: [15, 0.4039369144664876, 127, 1.5624683870098386, 2.6600176745055206, 32, 664, 62.640738279219]
Device Parameters: [2.2379605566929834, 7, 419, 31, 13]
Offload Decision: No


KeyboardInterrupt: 